# 📌 코드 작성 목적

- 프로젝트를 수행함에 있어 train 데이터셋의 크기가 개인 로컬 컴퓨터에서 다루기에는 용량이 매우 큰 편입니다.
- "Quick Draw!" 의 목적은 csv파일의 "word" 컬럼(label)의 이미지를 그린 획의 정보가 있는 "drawing" 컬럼의 정보를 사용해 이미지를 딥러닝 모델에 학습시켜, 특정 "word"를 맞추는 것입니다.
- 따라서, 효율적인 데이터 및 메모리 사용을 위해 필요한 정보만을 추출해서 병렬적으로 처리하기 위해 전체 데이터셋에서 필요한 컬럼들만 추출하여 100개의 파일로 chunk한 파일을 만듭니다.
    - 각각의 chunk 파일은 train 데이터셋에 잇는 340종의 이미지에 대한 csv파일별로 랜덤하지만 균일하게 정보를 추출해 담도록 했습니다.
    - 이렇게 생성된 100개의 `.gz` 파일을 사용해서 차후 모델에 chunk 파일에서 정보를 불러와서 이미지를 시각화해 모델에 학습시키는 데이터를 제공하는 데이터 파이프라인을 구축할 때 사용합니다.
- 전처리되어 경량화된 데이터를 모델에 입력하기 위한 파이프라인은 `[공통] data_pipeline(데이터 가져오기 및 시각화).ipynb`  파일에 구현해 정리하였습니다.

## STEP 1. 전처리 함수 정의

- 이 함수는 주어진 CSV 파일을 전처리하고, 지정된 라벨을 추가한 후, 데이터를 여러 개의 압축 파일로 분할 저장하는 작업을 수행합니다. 
- `pd.read_csv`를 사용하여 CSV 파일을 읽고, `pandas.DataFrame`에 라벨 열을 추가합니다. 
- 그 후, 데이터를 `divide_shuffles` 수만큼 분할하여 각각의 압축 파일(`gzip`)로 저장합니다. 
- 이 과정에서 `os`와 `gzip` 라이브러리가 파일 시스템 작업과 압축 파일 처리에 사용됩니다.
- 마지막으로, 원본 CSV 파일은 삭제되고, 처리된 파일의 이름은 로그 파일에 기록됩니다. 

In [ ]:
from tqdm import tqdm  # tqdm 라이브러리를 임포트하여 진행 상황을 시각화
import pandas as pd  # pandas 라이브러리를 데이터 처리를 위해 임포트
import os  # os 라이브러리를 운영 체제와 상호 작용을 위해 임포트
import gzip  # gzip 라이브러리를 압축 파일 처리를 위해 임포트

def preprocess_file(csv_file, raw_path, raw_shuffle_data_path, divide_shuffles, label, processed_files_log):
    try:
        # CSV 파일을 읽어서 데이터프레임으로 저장합니다.
        df = pd.read_csv(raw_path + csv_file, usecols=["drawing", "key_id"])

        # 라벨 열을 추가하고 지정된 "label" 값을 넣습니다.
        df["y"] = label

        # "key_id" 값을 사용하여 "cv" 열을 생성합니다.
        df["cv"] = (df.key_id // 10000) % divide_shuffles

        for k in range(divide_shuffles):
            # 새로운 파일 이름을 생성하여 압축 파일로 저장합니다.
            filename = raw_shuffle_data_path + f"train_k{k}.csv.gz"
            
            # "cv" 열 값이 "k"와 일치하는 행만 선택하여 "chunk"에 저장합니다.
            chunk = df[df.cv == k]
            
            # "key_id"와 "cv" 열을 삭제합니다.
            chunk = chunk.drop(["key_id", "cv"], axis=1)

            if not os.path.exists(filename):
                # 파일이 존재하지 않으면 새로 생성합니다.
                chunk.to_csv(filename, index=False, compression="gzip")
            else:
                with gzip.open(filename, "at") as f:
                    # 파일이 이미 있으면 gzip 모드에서 데이터를 추가로 저장합니다.
                    chunk.to_csv(f, header=False, index=False)

        os.remove(raw_path + csv_file)  # 원본 CSV 파일을 삭제합니다.

    except Exception as e:
        print(f"오류 발생: {csv_file} 처리 중 - {e}")  # 오류 발생 시 메시지 출력
        
    with open(processed_files_log, "a") as log:
        log.write(csv_file + "\n")  # 처리된 파일 로그에 기록합니다.

## STEP 2. 압축 해제 및 전처리

- 이 함수는 사용자로부터 데이터 전처리를 위한 경로 선택을 입력받아, 해당 경로에 있는 데이터를 전처리하는 과정을 수행합니다. 
- 사용자는 `train_raw` 또는 `train_simplified` 중 하나를 선택할 수 있습니다. 
- 선택된 경로에 따라 데이터 파일들을 읽고, 이를 셔플링 및 압축하여 저장하는 작업을 포함합니다. 
- 이 과정에서 `os`, `tqdm`, `gzip`, `pandas` 등의 라이브러리가 사용됩니다. 
- 주요 단계로는 사용자 입력 받기, 경로 설정, 디렉토리 생성, 파일 전처리 및 로깅, 그리고 최종적으로 각 파일을 셔플링 및 압축하여 저장하는 작업이 포함됩니다. 

- 또한 `~_processed_files.log` 파일에는 각각의 csv파일을 처리한 내용이 기록되며 이는 서버 연결이 끊기는 등의 이슈로 작업이 중단되어도 멈춘 부분부터 작업할 수 있도록 하기 위함 입니다.   

In [ ]:
def main():
    # 사용자 입력을 받아 전처리할 경로를 선택합니다.
    while True:
        print("전처리할 데이터 경로를 선택하세요:")
        print("1: train_raw")
        print("2: train_simplified")
        choice = input("선택 (1 또는 2): ")
        
        # 선택에 따라 경로를 설정합니다.
        if choice == '1':
            raw_path = "./data/train_raw/"
            processed_files_log = "./raw_processed_files.log"  # 1번을 선택한 경우 파일명 설정
            break
        elif choice == '2':
            raw_path = "./data/train_simplified/"
            processed_files_log = "./simplified_processed_files.log"  # 2번을 선택한 경우 파일명 설정
            break
        else:
            print("잘못된 선택입니다. 다시 입력해주세요.")

    # 설정된 경로 출력
    print(f"선택된 경로: {raw_path}")

    # 나머지 코드는 선택된 raw_path를 사용합니다.
    raw_shuffle_data_path = raw_path + 'shuffle_raw_gzs/'
    divide_shuffles = 100

    if not os.path.exists(raw_shuffle_data_path):
        os.makedirs(raw_shuffle_data_path)

    train_csvs = os.listdir(raw_path)
    for y, csv_file in tqdm(enumerate(train_csvs)):
        preprocess_file(csv_file, raw_path, raw_shuffle_data_path, divide_shuffles, y, processed_files_log)

    # 각 청크 파일에 대한 셔플링 및 압축 처리
    for k in tqdm(range(divide_shuffles)):
        filename = raw_shuffle_data_path + f'train_k{k}.csv.gz'
        with gzip.open(filename, 'rt') as f:
            df = pd.read_csv(f)
        
        df = df.sample(frac=1).reset_index(drop=True)
        with gzip.open(filename, 'wt') as f:
            df.to_csv(f, index=False)

if __name__ == "__main__":
    main()

### Zip파일 만들기(팀원 공유용)

- 이 코드는 원본 데이터를 압축 파일로 변환하는 과정을 담당합니다. 
    - 먼저, 원본 데이터가 위치한 디렉토리(`raw_path`)와 무작위로 섞인 데이터를 저장할 디렉토리(`raw_shuffle_data_path`)의 경로를 설정합니다. 
    - 그 후, `raw_shuffle_data_path` 디렉토리 내의 모든 `.gz` 파일을 찾아 리스트(`gz_files`)에 저장합니다. 
    - 이 파일들은 `zipfile.ZipFile`을 사용하여 `zip_filename`에 지정된 경로의 압축 파일로 저장됩니다. 
    - 각 파일이 압축 파일에 추가될 때마다 `tqdm` 라이브러리를 사용하여 진행 상황을 시각적으로 표시합니다.

In [ ]:
# 원본 데이터 디렉토리의 경로
raw_path = "./data/train_simplified/"

# 무작위로 섞인 원본 데이터를 저장할 디렉토리의 경로
raw_shuffle_data_path = raw_path + "shuffle_raw_gzs/"

# 압축 파일 경로를 지정
zip_filename = './train_simplified_chunked.zip'

# 폴더 내의 .gz 파일 목록을 가져옴
gz_files = [file for file in os.listdir(raw_shuffle_data_path) if file.endswith('.gz')]

# tqdm 진행 바를 설정
with tqdm(total=len(gz_files), unit="file") as pbar:
    pbar.set_description("압축 중")

    # zipfile 객체를 생성하여 파일을 하나씩 압축에 추가
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in gz_files:
            # 각 파일을 zip 파일에 추가
            zipf.write(os.path.join(raw_shuffle_data_path, file), arcname=file)
            # 진행 바 업데이트
            pbar.update(1)